In [1]:
from sentence_transformers.cross_encoder import CrossEncoder
import pandas as pd
import numpy as np

model = CrossEncoder("cross-encoder/stsb-distilroberta-base")

/home/cmusfel1/.local/lib/python3.11/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/home/cmusfel1/.local/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Create a statement from the complete question. Pretty much just subtracting the incorrect answer choices from "complete_question"

In [2]:
df = pd.read_csv("test-tail100-premises-bestof32.tsv", sep='\t')

def correct_statement(row):
    correct_answer = row['answer_key']
    complete_question = row['complete_question']
    
    start_pos = complete_question.find(f'({correct_answer})')
    end_pos = complete_question.find(' (', start_pos + 1)
    
    if end_pos == -1:
        end_pos = len(complete_question)
    
    return complete_question[:end_pos]

df['q_statement'] = df.apply(correct_statement, axis=1)

It is pretty sensitive so I ended up splitting corr_premises into two separate columns to compute the similarity score for each.

In [3]:
df['corr_premises'] = df['corr_premises'].str.strip("[]")
df['premise_split'] = df['corr_premises'].str.split("', '")

df['first_premise'] = df['premise_split'].apply(lambda x: x[0].strip("'"))
df['second_premise'] = df['premise_split'].apply(lambda x: x[1].strip("'") if len(x) > 1 else None)

df.drop(columns=['premise_split'], inplace=True)

Computing the similarity scores for each premise compared to the correct statment.

In [4]:
def first_similarity(row):
    query = row['q_statement']
    premise = row['first_premise']
    
    if premise is None:
        return np.nan
    
    sentence_combinations = [query, premise]
    score = model.predict(sentence_combinations)
    
    return score

def second_similarity(row):
    query = row['q_statement']
    premise = row['second_premise']
    
    if premise is None:
        return np.nan # really only useful for this one as some corr_premises never had a second premise
    
    sentence_combinations = [query, premise]
    score = model.predict(sentence_combinations)
    
    return score

df['first_similarity_score'] = df.apply(first_similarity, axis=1)
df['second_similarity_score'] = df.apply(second_similarity, axis=1)

In [5]:
df

,Unnamed: 0,id,question_stem,choices,complete_question,answer_key,foils,F1,F2,F3,corr_premises,F1_premises,corr_score,F1_score,q_statement,first_premise,second_premise,first_similarity_score,second_similarity_score
0,400,936,Animals died after the removal of a,(A) bush (B) street (C) house (D) city,Animals died after the removal of a (A) bush (...,A,"['B', 'C', 'D']",B,C,D,'The removal of a bush can cause animals to di...,['Streets are artificial environments that can...,3.599609,8.203125,Animals died after the removal of a (A) bush,The removal of a bush can cause animals to die.,Animals depend on bushes for food and shelter.,0.906569,0.398109
1,401,8-478,"If I want to go running at night, what can I u...",(A) A black shirt (B) Kitchen foil (C) Sunglas...,"If I want to go running at night, what can I u...",B,"['A', 'C', 'D']",A,C,D,"'Kitchen foil reflects light.', 'It can be use...",['A reflector can be used to increase visibili...,10.085938,2.796875,"If I want to go running at night, what can I u...",Kitchen foil reflects light.,It can be used as a reflector for running at n...,0.455466,0.510962
2,402,9-669,the closest star to our planet delivers solar ...,(A) maybe (B) all of these (C) this is sure (D...,the closest star to our planet delivers solar ...,C,"['A', 'B', 'D']",A,B,D,"'The sun is the closest star to Earth.', 'It d...","['The sun is the closest star to Earth.', 'It ...",0.736328,1.494141,the closest star to our planet delivers solar ...,The sun is the closest star to Earth.,It delivers solar energy to our planet.,0.724650,0.601800
3,403,7-732,Coal-fire power stations heat coal to incredib...,(A) produce energy (B) use heat energy (C) bur...,Coal-fire power stations heat coal to incredib...,A,"['B', 'C', 'D']",B,C,D,"'Coal-fire power stations produce energy.', 'T...",['Coal-fire power stations use heat energy to ...,1.373047,1.995117,Coal-fire power stations heat coal to incredib...,Coal-fire power stations produce energy.,They do this by burning coal to produce heat e...,0.737473,0.791122
4,404,7-658,Creatures sometimes have barbs on their backs ...,(A) wasp (B) bee (C) scorpion (D) butterfly,Creatures sometimes have barbs on their backs ...,D,"['A', 'B', 'C']",A,B,C,'Butterflies do not have barbs on their backs....,"['Wasps use barbs to sting their prey.', 'Barb...",0.928711,3.630859,Creatures sometimes have barbs on their backs ...,Butterflies do not have barbs on their backs.,Barbs are used for stinging and are found on w...,0.294434,0.797408
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,495,9-284,A person is heating water in order to cook pas...,(A) scalds (B) cools (C) toasts (D) freezes,A person is heating water in order to cook pas...,A,"['B', 'C', 'D']",B,C,D,'Water that is heated to a high temperature is...,"['When water is heated, it loses heat and cool...",2.386719,4.148438,A person is heating water in order to cook pas...,Water that is heated to a high temperature is ...,Scalding water can cause burns.,0.681857,0.545094
96,496,7-1186,Pasta may be cooked in water when,(A) the water is warm (B) the water is on the ...,Pasta may be cooked in water when (A) the wate...,C,"['A', 'B', 'D']",A,B,D,"'Water bubbles when it is heated.', 'This caus...",['Pasta is cooked in water when the water is w...,2.138672,3.261719,Pasta may be cooked in water when (A) the wate...,Water bubbles when it is heated.,This causes the pasta to cook.,0.579239,0.714890
97,497,926,A decrease in diseases,(A) has no impact on a population (B) leads to...,A decrease in diseases (A) has no impact on a ...,C,"['A', 'B', 'D']",A,B,D,'Decreases in diseases can lead to less sick p...,['A decrease in diseases does not impact a pop...,5.878906,2.964844,A decrease in diseases (A) has no impact on a ...,Decreases in diseases can lead to less sick pe...,Less sick people means fewer emergency room vi...,0.677256,0.342784
98,498,7-519,"When soil is viewed in a scientific way, what ...",(A) insects like big beetle